Start by importing pandas and fpsnippets for the zip codes in my data. 

In [458]:
import pandas as pd
import fpsnippets

watertoxdata = pd.read_csv("WaterToxins.csv")

Dataset is for Alleghany County, so we first sort out everything besides the city of Pittsburgh. We also sort out all the columns that are relevant. 

In [460]:
filtered_data = watertoxdata[watertoxdata['CITY_NAME'].str.contains('PITTSBURGH', case=False, na=False)]
columns_to_display = ['FACILITY_NAME', 'STREET_ADDRESS',  'CITY_NAME', 'ZIP_CODE', 'CHEM_NAME', 'CARCINOGEN', 'CLEAN_AIR', 'TOTAL_RELEASE']
filtered_data[columns_to_display].head()

,FACILITY_NAME,STREET_ADDRESS,CITY_NAME,ZIP_CODE,CHEM_NAME,CARCINOGEN,CLEAN_AIR,TOTAL_RELEASE
0,PRUETT-SCHAFFER CHEMICAL CO INC.,3327 STAFFORD ST,PITTSBURGH,15204,Xylene (mixed isomers),N,Y,NaN
3,E.E. ZIMMERMAN CO,2020 KNOTT ST,PITTSBURGH,15233,Methyl ethyl ketone,N,N,NaN
4,E.E. ZIMMERMAN CO,2020 KNOTT ST,PITTSBURGH,15233,Methanol,N,Y,NaN
5,E.E. ZIMMERMAN CO,2020 KNOTT ST,PITTSBURGH,15233,"1,2,4-Trimethylbenzene",N,N,NaN
6,E.E. ZIMMERMAN CO,2020 KNOTT ST,PITTSBURGH,15233,Toluene,N,Y,NaN


Here we sort the data and figure out the zip code with the highest total release of chemicals (per lb) into the water. The majority is from facilities in 15225 which encompasses the Neville Township. 

In [464]:
filtered_data = filtered_data.dropna(subset=['TOTAL_RELEASE'])

filtered_data = filtered_data[filtered_data['ZIP_CODE'] != 152251516]
filtered_data = filtered_data[filtered_data['ZIP_CODE'] != 152042599]
filtered_data = filtered_data[filtered_data['ZIP_CODE'] != 152206246]
filtered_data = filtered_data[filtered_data['ZIP_CODE'] != 152342624]


filtered_data = filtered_data.sort_values(by='TOTAL_RELEASE', ascending=False)
filtered_data[columns_to_display].head()


,FACILITY_NAME,STREET_ADDRESS,CITY_NAME,ZIP_CODE,CHEM_NAME,CARCINOGEN,CLEAN_AIR,TOTAL_RELEASE
10330,SHENANGO INC,200 NEVILLE RD,PITTSBURGH,15225,Ammonia,N,N,197243.0
13890,SHENANGO INC,200 NEVILLE RD,PITTSBURGH,15225,Ammonia,N,N,160995.0
8662,LTV STEEL COMPANY INC PITTSBURGH WORKS,4650 SECOND AVENUE,PITTSBURGH,15207,Ammonia,N,N,150000.0
13488,SHENANGO INC,200 NEVILLE RD,PITTSBURGH,15225,Ammonia,N,N,132370.0
12201,SHENANGO INC,200 NEVILLE RD,PITTSBURGH,15225,Ammonia,N,N,92000.0


We group together the total releases of each zipcode

In [466]:
zip_code_totals = filtered_data.groupby('ZIP_CODE')['TOTAL_RELEASE'].sum()

We find the zip codes with the most total releases (worst ones to live at)

In [470]:
worst_zip_codes = zip_code_totals.sort_values(ascending=False).head(5)
worst_zip_codes

ZIP_CODE
15225    1542450.60
15207     771586.00
15201     139734.71
15236         48.00
15226         21.00
Name: TOTAL_RELEASE, dtype: float64

In [471]:
zip_to_name = {
    '15225': "Neville",
    '15207': 'Greenfield, New Homestead, Lincoln Place, Glen Hazel, Hazelwood, Hays',
    '15201': 'Central Lawrenceville, Lower Lawrenceville, Upper Lawrenceville, Stanton Heights, Polish Hill',
    '15236': 'Pleasant HIlls, South Park, Whitehall, Baldwin',
    '15226': 'Brookline, Overbrook, Beechview, Bon Air, Baldwin'
}

zip_to_name = {int(i): v for i, v in zip_to_name.items()}

worst_neighborhoods = worst_zip_codes.rename(index=zip_to_name)

worst_neighborhoods

ZIP_CODE
Neville                                                                                          1542450.60
Greenfield, New Homestead, Lincoln Place, Glen Hazel, Hazelwood, Hays                             771586.00
Central Lawrenceville, Lower Lawrenceville, Upper Lawrenceville, Stanton Heights, Polish Hill     139734.71
Pleasant HIlls, South Park, Whitehall, Baldwin                                                        48.00
Brookline, Overbrook, Beechview, Bon Air, Baldwin                                                     21.00
Name: TOTAL_RELEASE, dtype: float64

Here are the top zipcodes with no releases into the water at all. 

In [473]:

top_zip_codes = zip_code_totals.sort_values(ascending=True).head(5)
top_zip_codes

ZIP_CODE
15204     0.0
15219     0.0
15233     0.0
15275     0.0
15226    21.0
Name: TOTAL_RELEASE, dtype: float64

Now as neighborhoods

In [477]:
zip_name = {
    '15204': 'Crafton Heights, Windgap, Chartiers, Sheraden, Esplen',
    '15219': 'Market Square, Cultural District, The Point, West Oakland, Crawford-Roberts',
    '15233': 'Central Northside, Marshall-Shadeland, Allegheny West, Manchester',
    '15275': 'North Fayette',
    '15226': 'Brookline, Overbrook, Beechview, Bon Air, Baldwin'
}

zip_name = {int(i): v for i, v in zip_name.items()}
top_neighborhoods = top_zip_codes.rename(index=zip_name)


ZIP_CODE
Crafton Heights, Windgap, Chartiers, Sheraden, Esplen                           0.0
Market Square, Cultural District, The Point, West Oakland, Crawford-Roberts     0.0
Central Northside, Marshall-Shadeland, Allegheny West, Manchester               0.0
North Fayette                                                                   0.0
Brookline, Overbrook, Beechview, Bon Air, Baldwin                              21.0
Name: TOTAL_RELEASE, dtype: float64